# Data Analytics I: Assignment 3

imports

In [1]:
import pandas as pd
import numpy as np

from itertools import combinations
import itertools

## Data Preprocessing

### 1. Forming transactional dataset

In [2]:
df = pd.read_csv("../ml-latest-small/ratings.csv")

df_filtered = df[df["rating"] > 2]

user_movie_counts = df_filtered.groupby("userId").size()
valid_users = user_movie_counts[user_movie_counts > 10].index

df_valid_users = df_filtered[df_filtered["userId"].isin(valid_users)]

transactional_data = (
    df_valid_users.groupby("userId")["movieId"].apply(set).reset_index()
)

In [3]:
transactional_data.to_csv("transactional_data.csv", index=False)

### 2. Train-test split

In [4]:
def split_movies(movies):
    movies = np.array(list(movies))
    np.random.shuffle(movies)
    split_idx = int(len(movies) * 0.8)
    train_movies = movies[:split_idx]
    test_movies = movies[split_idx:]
    return set(train_movies), set(test_movies)

In [5]:
train_data = []
test_data = []

for index, row in transactional_data.iterrows():
    user_id = row['userId']
    movies = row['movieId']
    
    train_movies, test_movies = split_movies(movies)
    
    train_data.append({'userId': user_id, 'movieId': train_movies})
    
    if test_movies:
        test_data.append({'userId': user_id, 'movieId': test_movies})

train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

In [6]:
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

## Association Rule Mining

### 1. Apriori Algorithm

In [7]:
def generate_candidates(Lk, k):
    """
    Generate candidate itemsets of size k+1 from frequent itemsets of size k.
    """
    candidates = set()
    Lk_list = list(Lk)
    
    for i in range(len(Lk_list)):
        for j in range(i + 1, len(Lk_list)):
            # Join step: create k+1 candidate itemset by combining two k-itemsets
            candidate = Lk_list[i].union(Lk_list[j])
            if len(candidate) == k + 1:
                candidates.add(candidate)
    
    return candidates


def prune_candidates(Ck, Lk, k):
    """
    Prune the candidate set Ck by removing itemsets where any (k-1)-subset is not in Lk.
    """
    
    pruned_candidates = set()
    for candidate in Ck:
        all_subsets_frequent = True
        for subset in combinations(candidate, k):
            if frozenset(subset) not in Lk:
                all_subsets_frequent = False
                break
        if all_subsets_frequent:
            pruned_candidates.add(candidate)

    return pruned_candidates


def get_frequent_itemsets(transactions, candidates, minsup):
    """
    Count the support of candidates and return the frequent itemsets.
    """

    candidate_counts = {candidate: 0 for candidate in candidates}
    
    # Count occurrences of each candidate in the transaction set
    for transaction in transactions:
        for candidate in candidates:
            if candidate.issubset(transaction):
                candidate_counts[candidate] += 1
    
    # Filter candidates by minimum support
    total_transactions = len(transactions)
    frequent_itemsets = {candidate for candidate, count in candidate_counts.items() if count / total_transactions >= minsup}
    
    return frequent_itemsets


In [8]:
def apriori(transactions, min_support):
    """
    Apriori algorithm to find frequent itemsets.
    transactions: list of transactions (each transaction is a set of items)
    min_support: minimum support threshold
    """
    # Step 1: Find frequent 1-itemsets
    item_support = {}
    for transaction in transactions:
        for item in transaction:
            if item not in item_support:
                item_support[item] = 0
            item_support[item] += 1

    total_transactions = len(transactions)
    L1 = {
        frozenset([item])
        for item, count in item_support.items()
        if count / total_transactions >= min_support
    }

    L = [L1]
    k = 1

    while L[k - 1]:
        # Step 2: Generate candidate itemsets of size k+1
        Ck = generate_candidates(L[k - 1], k)

        # Step 3: Prune candidate itemsets
        Ck = prune_candidates(Ck, L[k - 1], k)

        # Step 4: Get frequent itemsets from the candidate set
        Lk = get_frequent_itemsets(transactions, Ck, min_support)

        if Lk:
            L.append(Lk)
            k += 1
        else:
            break

    return set().union(*L) if L else set()

In [9]:
def calculate_support(itemset, transactions):
    """
    Calculate the support count of an itemset in the transactions.
    Returns the support as a fraction (support count / total_transactions).
    """
    total_transactions = len(transactions)
    count = sum(1 for transaction in transactions if itemset.issubset(transaction))
    return count / total_transactions  

In [10]:
def generate_rules(frequent_itemsets, transactions, min_conf):
    """
    Generate association rules from frequent itemsets.
    frequent_itemsets: set of frequent itemsets
    transactions: list of transactions (each transaction is a set of items)
    min_conf: minimum confidence threshold
    """
    rules = []
    
    for itemset in frequent_itemsets:
        itemset_list = list(itemset)
        for i in range(1, len(itemset_list)):  # Generate non-empty subsets of size 1 to len(itemset)-1
            for subset in combinations(itemset_list, i):
                subset = frozenset(subset)
                remaining = itemset - subset
                if remaining:
                    support_itemset = calculate_support(itemset, transactions)  # Get support as fraction
                    support_subset = calculate_support(subset, transactions)  # Get support as fraction
                    
                    confidence = support_itemset / support_subset
                    
                    if confidence >= min_conf:
                        rule = (subset, remaining, confidence, support_itemset)
                        rules.append(rule)
    
    return rules

In [11]:
transactions = [set(movies) for movies in train_df['movieId']]

In [12]:
min_support = 0.05

frequent_itemsets = apriori(transactions, min_support)

print(f"Frequent Itemsets: {frequent_itemsets}")

Frequent Itemsets: {frozenset({1036, 1198}), frozenset({1193, 2571}), frozenset({5952, 89745}), frozenset({1291, 260, 541}), frozenset({5952, 7153, 260, 318}), frozenset({1196, 260, 5989}), frozenset({4306, 780}), frozenset({588, 5349}), frozenset({377, 589, 110}), frozenset({8360, 1210}), frozenset({480, 2028, 110}), frozenset({480, 316, 165}), frozenset({3147, 2028, 260}), frozenset({7361, 47}), frozenset({316, 231}), frozenset({296, 2683}), frozenset({32, 357}), frozenset({2762, 4262}), frozenset({608, 260, 1196, 318}), frozenset({2329, 1210, 1213}), frozenset({608, 3578, 1213}), frozenset({296, 595, 318}), frozenset({2858, 2571, 260}), frozenset({4993, 2571, 589, 2959}), frozenset({7153, 4306}), frozenset({5418, 5445, 4226}), frozenset({5952, 260, 3578, 1198}), frozenset({296, 1, 589}), frozenset({1136, 500}), frozenset({7153, 260, 2959}), frozenset({110, 588, 316}), frozenset({5952, 6377, 1580}), frozenset({1198, 1213, 1214}), frozenset({5952, 4993, 2858}), frozenset({380, 1196, 1

In [13]:
min_conf = 0.7

rules = generate_rules(frequent_itemsets, transactions, min_conf)

for rule in rules:
    antecedent, consequent, confidence, support = rule
    print(f"Rule: {set(antecedent)} -> {set(consequent)}, Confidence: {confidence:.2f}, Support: {support:.2f}")

Rule: {5952, 260, 318} -> {7153}, Confidence: 0.78, Support: 0.06
Rule: {7153, 260, 318} -> {5952}, Confidence: 0.80, Support: 0.06
Rule: {1196, 5989} -> {260}, Confidence: 0.73, Support: 0.06
Rule: {260, 5989} -> {1196}, Confidence: 0.81, Support: 0.06
Rule: {316, 165} -> {480}, Confidence: 0.74, Support: 0.05
Rule: {3147, 260} -> {2028}, Confidence: 0.73, Support: 0.05
Rule: {608, 1196, 318} -> {260}, Confidence: 0.72, Support: 0.05
Rule: {1210, 1213} -> {2329}, Confidence: 0.70, Support: 0.05
Rule: {3578, 1213} -> {608}, Confidence: 0.73, Support: 0.05
Rule: {2858, 260} -> {2571}, Confidence: 0.70, Support: 0.09
Rule: {4993, 2571, 589} -> {2959}, Confidence: 0.70, Support: 0.05
Rule: {4993, 589, 2959} -> {2571}, Confidence: 0.86, Support: 0.05
Rule: {5418, 5445} -> {4226}, Confidence: 0.76, Support: 0.05
Rule: {5952, 3578, 260} -> {1198}, Confidence: 0.81, Support: 0.06
Rule: {5952, 3578, 1198} -> {260}, Confidence: 0.81, Support: 0.06
Rule: {3578, 260, 1198} -> {5952}, Confidence: 

### 2. Recommendation

In [14]:
def top_rules_by_metric(rules, metric, top_n=100):
    """
    Get the top N rules sorted by a specified metric.
    metric_index: index of the metric to sort by (2 for confidence, 3 for support)
    top_n: number of top rules to return
    """
    if metric == 'support':
        metric_index = 3
    elif metric == 'confidence':
        metric_index = 2

    sorted_rules = sorted(rules, key=lambda x: x[metric_index], reverse=True)
    return sorted_rules[:top_n]

top 100 association rules based on support and confidence

In [15]:
top_100_support = top_rules_by_metric(rules, 'support') 
top_100_confidence = top_rules_by_metric(rules, 'confidence')

team_id = 51

with open(f'{team_id}_top100RulesBySup.txt', 'w') as f:
    for rule in top_100_support:
        antecedent, consequent, confidence, support = rule
        f.write(f"Rule: {set(antecedent)} -> {set(consequent)}, Confidence: {confidence:.2f}, Support: {support:.2f}\n")

with open(f'{team_id}_top100RulesByConf.txt', 'w') as f:
    for rule in top_100_confidence:
        antecedent, consequent, confidence, support = rule
        f.write(f"Rule: {set(antecedent)} -> {set(consequent)}, Confidence: {confidence:.2f}, Support: {support:.2f}\n")

rules that appear in both lists, arranged based on their confidence score

In [16]:
rules_support_set = set((rule[0], rule[1]) for rule in top_100_support)
rules_confidence_set = set((rule[0], rule[1]) for rule in top_100_confidence)

shared_rules = rules_support_set.intersection(rules_confidence_set)
filtered_shared_rules = [rule for rule in top_100_confidence if (rule[0], rule[1]) in shared_rules]
sorted_shared_rules = sorted(filtered_shared_rules, key=lambda x: x[2], reverse=True)

for rule in sorted_shared_rules:
    antecedent, consequent, confidence, support = rule
    print(f"Rule: {set(antecedent)} -> {set(consequent)}, Confidence: {confidence:.2f}, Support: {support}")